In [ ]:
!pip -q install langchain==0.3.0 langchain-community==0.3.0 langchain-openai==0.2.0 \
                chromadb openai tiktoken

import os
os.environ["OPENAI_API_KEY"] = "dummy"


In [62]:
import json, os
from langchain.docstore.document import Document

VERT_JSONL_PATH = "/content/drive/MyDrive/VERT.json"

def load_vert_rows(path):
    rows=[]
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                try: rows.append(json.loads(line))
                except: pass
    return rows

rows = load_vert_rows(VERT_JSONL_PATH)

def rows_to_docs(rows, max_chars=1200):
    docs=[]
    for r in rows:
        code = (r.get("Code") or "").strip()
        text = f"CODE:\n{code[:max_chars]}"
        docs.append(Document(page_content=text, metadata={}))
    return docs

docs = rows_to_docs(rows)
print(f"Prepared {len(docs)} docs (truncated).")


Prepared 20000 docs (truncated).


In [63]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

emb = OpenAIEmbeddings(model="text-embedding-3-small")

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
docs_split = splitter.split_documents(docs)

vstore = Chroma(collection_name="vert_sva_gen", embedding_function=emb)

BATCH = 128
for i in range(0, len(docs_split), BATCH):
    vstore.add_documents(docs_split[i:i+BATCH])


retriever = vstore.as_retriever(search_kwargs={"k": 2})


Total chunks after split: 20000
Vector store ready.


In [ ]:

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import json as pyjson

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    response_format={"type": "json_object"},
)

SYS = """You generate strict SystemVerilog Assertions from a requirement and signals.
Return only JSON with fields: assertion, anti_vacuity_cover, notes.

HARD RULES:
- assertion: include a NAMED property + `assert property (NAME);`
- anti_vacuity_cover: must be a `cover property (...)` (never assert)
- Both properties: `@(posedge {clock}) disable iff (!{reset})`
- For (valid && !ready) backpressure, use overlapped `|->` so stability starts same cycle.
- Use $stable(signal) / $past(signal). NEVER write `data == $stable(data)`.
- Use `until` / `until_with` to hold until release; use `until_with` to include the release cycle.
- Cover must witness: antecedent occurs, then release later (e.g. `(valid && !ready) ##[1:$] ready`).
- Do not copy exemplars verbatim; they’re hints only.
"""

USR = """CLOCK: {clock}
RESET: {reset}
SIGNALS: {signals}
REQUIREMENT: {requirement}

RETRIEVED EXEMPLARS :
{context}
"""

prompt = ChatPromptTemplate.from_messages([("system", SYS), ("user", USR)])
chain = prompt | llm

def generate_sva(requirement: str,
                 signals: dict,
                 clock: str = "clk",
                 reset: str = "rst_n",
                 k: int = 30):
    query = requirement + " " + " ".join(signals.keys())
    hits = retriever.invoke(query)
    context = "\n\n---\n".join([h.page_content for h in hits])
    signallist = ", ".join([f"{k}={v}" for k, v in signals.items()])

    res = chain.invoke({
        "clock": clock,
        "reset": reset,
        "signals": signallist,
        "requirement": requirement,
        "context": context
    })
    out = pyjson.loads(res.content)
    assert "assertion" in out and "anti_vacuity_cover" in out and "notes" in out
    assert "assert property" in out["assertion"]
    assert "cover property" in out["anti_vacuity_cover"]
    return out


/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  if (await self.run_code(code, result,  async_=asy)):


In [65]:
def display_sva(out: dict):
    """Pretty-print the SVA generator output dict."""
    a = (out.get("assertion") or "").strip()
    c = (out.get("anti_vacuity_cover") or "").strip()
    n = (out.get("notes") or "").strip()
    line = "-" * 60

    print(f"{line}\nASSERTION\n{line}\n{a}\n")
    print(f"{line}\nANTI-VACUITY COVER\n{line}\n{c}\n")
    if n:
        print(f"{line}\nNOTES\n{line}\n{n}\n")




In [66]:
resp = generate_sva(
    requirement="When valid is high and ready is low, data must remain stable until ready goes high.",
    signals={"valid":"valid","ready":"ready","data":"data[7:0]"},
    clock="clk", reset="rst_n"
)
display_sva(resp)


------------------------------------------------------------
ASSERTION
------------------------------------------------------------
property data_stability; @(posedge clk) disable iff (!rst_n) (valid && !ready) |-> $stable(data); assert property (data_stability);

------------------------------------------------------------
ANTI-VACUITY COVER
------------------------------------------------------------
cover property (valid && !ready |-> (##[1:$] ready));

------------------------------------------------------------
NOTES
------------------------------------------------------------
This assertion ensures that when valid is high and ready is low, the data remains stable until ready goes high.

